In [7]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')

!unzip "/content/drive/MyDrive/train.zip"


train_path = '/content/train'


Streaming output truncated to the last 5000 lines.
 extracting: train/neutral/im846.png  
 extracting: train/neutral/im847.png  
 extracting: train/neutral/im848.png  
 extracting: train/neutral/im849.png  
 extracting: train/neutral/im85.png  
 extracting: train/neutral/im850.png  
 extracting: train/neutral/im851.png  
 extracting: train/neutral/im852.png  
 extracting: train/neutral/im853.png  
 extracting: train/neutral/im854.png  
 extracting: train/neutral/im855.png  
 extracting: train/neutral/im856.png  
 extracting: train/neutral/im857.png  
 extracting: train/neutral/im858.png  
 extracting: train/neutral/im859.png  
 extracting: train/neutral/im86.png  
 extracting: train/neutral/im860.png  
 extracting: train/neutral/im861.png  
 extracting: train/neutral/im862.png  
 extracting: train/neutral/im863.png  
 extracting: train/neutral/im864.png  
 extracting: train/neutral/im865.png  
 extracting: train/neutral/im866.png  
 extracting: train/neutral/im867.png  
 extracting: tr

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2 as cv  
import tensorflow as tf  
from keras.preprocessing.image import ImageDataGenerator  
from tensorflow.keras.models import load_model  
from keras.models import Sequential  
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten  
from keras.optimizers import Adam  
from keras.layers import BatchNormalization  
from keras.regularizers import l2  
from keras.callbacks import ReduceLROnPlateau, EarlyStopping 
from tensorflow.keras import optimizers

import warnings  

In [ ]:
train_path = '/content/train'

contents = os.listdir(train_path)

In [ ]:
original_dataset_dir = r"/content/train"
train_dir = 'CK+48_train'
test_dir = 'CK+48_test'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

train_ratio = 0.8

for emotion in os.listdir(original_dataset_dir):
    emotion_dir = os.path.join(original_dataset_dir, emotion)
    if os.path.isdir(emotion_dir):
        images = [f for f in os.listdir(emotion_dir) if os.path.isfile(os.path.join(emotion_dir, f))]

        np.random.shuffle(images)

        train_size = int(len(images) * train_ratio)
        train_images = images[:train_size]
        test_images = images[train_size:]

        train_emotion_dir = os.path.join(train_dir, emotion)
        test_emotion_dir = os.path.join(test_dir, emotion)
        os.makedirs(train_emotion_dir, exist_ok=True)
        os.makedirs(test_emotion_dir, exist_ok=True)

        for image in train_images:
            shutil.copy(os.path.join(emotion_dir, image), os.path.join(train_emotion_dir, image))
        for image in test_images:
            shutil.copy(os.path.join(emotion_dir, image), os.path.join(test_emotion_dir, image))

print("Dataset splitting complete")

In [ ]:
img = cv2.imread(r'/content/train/Sad/0.jpg')

height, width, channels = img.shape
print(f'Dimensions: {width}x{height}')

In [ ]:
train_data_generator = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=15, 
    width_shift_range=0.15,  
    height_shift_range=0.15,  
    shear_range=0.15,  
    zoom_range=0.15,  
    horizontal_flip=True,  
)


fer_training_data = train_data_generator.flow_from_directory(
    r'/content/CK+48_train',
    target_size=(48, 48),  
    batch_size=64,  
    color_mode='grayscale',  
    class_mode='categorical' 
)

fer_training_data

In [ ]:
test_data_generator = ImageDataGenerator(rescale=1./255)

fer_test_data = test_data_generator.flow_from_directory(
    r'/content/CK+48_test',  
    target_size = (48, 48),  
    batch_size = 64,  
    color_mode = 'grayscale', 
    class_mode = 'categorical' 
)

fer_test_data

In [ ]:
from tensorflow.keras import optimizers

optims = [
    optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam'),
    optimizers.Adam(0.001),
]

In [ ]:
+

In [ ]:
# Reduce learning rate when a metric has stopped improving
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [ ]:
batch_size = 64
history = model.fit(
    fer_training_data,
    epochs=60,
    validation_data=fer_test_data,
    batch_size = 64,
    callbacks=callbacks,
)

In [ ]:
model.save('/content/drive/My Drive/my_model.h5')
